In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# pip install datasets transformers

# 1.dataset load
huggingface의 모델을 쉽게 학습시키기 위해 제공되는 기능을 사용해 dataset dictonary형태로 만들어준다.

In [ ]:
import torch

In [ ]:
from datasets import load_dataset,ReadInstruction
data_files = {"train": "/content/drive/MyDrive/train2.csv", "validation": "/content/drive/MyDrive/validation2.csv"}
dataset = load_dataset('csv', data_files=data_files)

In [ ]:
dataset

## 1-1.pretrained tokenizer load

In [ ]:
# from transformers import PreTrainedTokenizerFast, BartModel

# tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')
# model = BartModel.from_pretrained('gogamza/kobart-base-v2')

In [ ]:
from transformers import PreTrainedTokenizerFast, BartModel

tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')

## 1-2.학습을 위한 추가 전처리
kobart모델은 문장 시작과 뒤에 bos와 eos가 필요하다.

In [ ]:
bos = "<s>"
eos = "</s>"
def preprocess_function(examples):
    inputs = [bos + doc + eos for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
tokenized_dataset

# 2.kobart 기반의 summarization task를 가지고 있는 사전학습 모델을 불러온다.

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
from transformers import BartForConditionalGeneration
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

# 2-1. 추가된 custom dataset을 넣어 학습한다.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    fp16=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [ ]:
trainer.train()

# 2-2.모델 저장

In [ ]:
test_model = trainer.model.cuda()

In [ ]:
torch.save(test_model, '/content/drive/MyDrive/model.pt')

# 3.시연

In [ ]:
text = """
1일 오후 9시까지 최소 20만3220명이 코로나19에 신규 확진됐다. 또다시 동시간대 최다 기록으로, 사상 처음 20만명대에 진입했다.
방역 당국과 서울시 등 각 지방자치단체에 따르면 이날 0시부터 오후 9시까지 전국 신규 확진자는 총 20만3220명으로 집계됐다.
국내 신규 확진자 수가 20만명대를 넘어선 것은 이번이 처음이다.
동시간대 최다 기록은 지난 23일 오후 9시 기준 16만1389명이었는데, 이를 무려 4만1831명이나 웃돌았다. 전날 같은 시간 기록한 13만3481명보다도 6만9739명 많다.
확진자 폭증은 3시간 전인 오후 6시 집계에서도 예견됐다.
오후 6시까지 최소 17만8603명이 신규 확진돼 동시간대 최다 기록(24일 13만8419명)을 갈아치운 데 이어 이미 직전 0시 기준 역대 최다 기록도 넘어섰다. 역대 최다 기록은 지난 23일 0시 기준 17만1451명이었다.
17개 지자체별로 보면 서울 4만6938명, 경기 6만7322명, 인천 1만985명 등 수도권이 12만5245명으로 전체의 61.6%를 차지했다. 서울과 경기는 모두 동시간대 기준 최다로, 처음으로 각각 4만명과 6만명을 넘어섰다.
비수도권에서는 7만7975명(38.3%)이 발생했다. 제주를 제외한 나머지 지역에서 모두 동시간대 최다를 새로 썼다.
부산 1만890명, 경남 9909명, 대구 6900명, 경북 6977명, 충남 5900명, 대전 5292명, 전북 5150명, 울산 5141명, 광주 5130명, 전남 4996명, 강원 4932명, 충북 3845명, 제주 1513명, 세종 1400명이다.
집계를 마감하는 자정까지 시간이 남아있는 만큼 2일 0시 기준으로 발표될 신규 확진자 수는 이보다 더 늘어날 수 있다. 이에 따라 최종 집계되는 확진자 수는 21만명 안팎을 기록할 수 있을 전망이다.
한편 전날 하루 선별진료소에서 이뤄진 검사는 70만8763건으로 검사 양성률은 40.5%다. 양성률이 40%를 넘은 것은 이번이 처음이다. 확산세가 계속 거세질 수 있다는 얘기다.
이날 0시 기준 신규 확진자는 13만8993명이었다. 이틀 연속 13만명대를 이어갔다.
"""

text = text.replace('\n', ' ')

raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

In [ ]:
# device = torch.device("cuda")
# summary_ids = t_model.generate(torch.tensor([input_ids], dtype=torch.int32, device=device),  num_beams=4,  max_length=512,  eos_token_id=1)
# tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

In [ ]:
device = torch.device("cuda")
summary_ids = test_model.generate(torch.tensor([input_ids], dtype=torch.int32, device=device),  num_beams=4,  max_length=512,  eos_token_id=1)
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

# 4.저장한 모델 load 및 시연

In [ ]:
t_model = torch.load('/content/drive/MyDrive/model.pt')

In [ ]:
text2 = "한편, 2012년부터 명문대학들이 중심이 되어 본격적으로 촉발된 MOOCs(Massive Open Online Courses)가 지속적인 성장을 하면서 고등교육발 혁신이 지속되고 있으며, 칸 아카데미(Kahn Academy)와 같은 무료 온라인 자원이 지속적으로 확충됨에 따라 디지털 환경에서의 교수학습 방법이 구체적으로 진화하고 있다. 특히 가정에서 수업내용을 온라인으로 수강하고 학교에서 다양한 활동을 통해 지식을 심화시키는 “flipped classroom” 등의 방법이 소개되기도 하였다. 다만, 미국교육의 전통과 환경이 우리나라에서 최근 중요하게 강조하는 학습자 중심, 협업, 참여 등의 활동에 오랜 기간 동안 이미 익숙해 있었기 때문에, 이들 디지털 도구와 자원은 방법론의 진화 수준으로 받아들이는 측면이 강하다고 볼 수 있다."

In [ ]:
raw_input_ids = tokenizer.encode(text2)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

In [ ]:
device = torch.device("cuda")
summary_ids = t_model.generate(torch.tensor([input_ids], dtype=torch.int32, device=device),  num_beams=4,  max_length=512,  eos_token_id=1)
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

In [ ]:
text3 = """'유전자'의 속성에 대해 가설을 제
시하는 것은 아직까지는 의미가 없다." 그레고어 멘델 Gegor Mendel은 녹
색 콩과 노란색 콩을 수년 동안 연구하면서 유전자 같은 것이 틀림없
이 존재함을 증명했다. 색상을 비롯한 다른 특성들은 온도와 토질 같
은 많은 요소들에 따라 달라진다. 하지만 '어떤 것'은 온전히 보존되는
데, 혼합되거나 분산되지 않으며, 틀림없이 정량화된다." 멘델은 정확
하게 명명하지는 않았지만 유전자를 발견한 것이다.
"""

In [ ]:
text3 = text3.replace('\n', ' ')

In [ ]:
raw_input_ids = tokenizer.encode(text3)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

In [ ]:
device = torch.device("cuda")
summary_ids = t_model.generate(torch.tensor([input_ids], dtype=torch.int32, device=device),  num_beams=4,  max_length=142,  eos_token_id=1)
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)